In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
sys.path.append('../../reproducible_figures/scripts/')
import rep_fig_vis as rfv
sys.path.append('../scripts/')
from loadpaths_glc import loadpaths
import data_loading_utils as dlu
import glc_prediction_utils as gpu
path_dict = loadpaths()

In [7]:
dict_dfs, dict_dfs_species, dict_val_species = dlu.load_metadata(create_geo=True, add_h3=False,
                                               create_validation_set=True)
# type(dict_dfs['df_test_pa'])

Dropped PO data because takes ages with geometry
Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2', 'publisher', 'month', 'day', 'region', 'dayOfYear', 'country']
Dropped 49637/1483637 duplicates in df_train_pa_species
Created validation set with 8898 entries


In [4]:
val_or_test = 'val'

dict_dfs, dict_dfs_species, dict_val_species = dlu.load_metadata(create_geo=True, add_h3=False,
                                create_validation_set=True if val_or_test == 'val' else False)
tmp = gpu.predict_using_buffer(dict_dfs=dict_dfs, dict_dfs_species=dict_dfs_species,
                            save_pred=True, method=f'lc-specific', buffer_deg=0.2,
                            eval_mode=val_or_test)
if val_or_test == 'val':
    f1_score = gpu.compute_f1_score_dicts(dict_val=dict_val_species, dict_pred=tmp)
    print(f'F1 score: {f1_score}')

Dropped PO data because takes ages with geometry
Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2', 'publisher', 'month', 'day', 'region', 'dayOfYear', 'country']
Dropped 49637/1483637 duplicates in df_train_pa_species
Created validation set with 8898 entries
    LandCover  count
0         0.0     25
1         1.0     15
2         2.0     18
3         4.0     22
4         5.0     16
5         8.0     20
6         9.0     16
7        10.0     19
8        11.0     14
9        12.0     17
10       13.0     16
11       14.0     17
12       15.0     11
13       16.0     11
14       17.0     13


100%|██████████| 8898/8898 [00:52<00:00, 169.96it/s]


Predictions done (8897 total). No nearby points: 0.
F1 score: 0.3097099169950371


## Predictions

In [4]:
(df_train, df_val, df_test), _ = dlu.create_full_pa_ds()

Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2', 'publisher', 'month', 'day', 'region', 'dayOfYear', 'country']
Dropped 49637/1483637 duplicates in df_train_pa_species
Dropped 135101/5079797 duplicates in df_train_po_species
Created validation set with 8898 entries


In [5]:
# df_train.dropna(inplace=True)
df_train

,lng,lat,year,Elevation,LandCover,Bio1,Bio2,Bio3,Bio4,Bio5,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,3.099038,43.134956,2021,47.0,8.0,2883,68,3,5545,3007,...,2958,2816,6355,933,143,395,2282,770,770,1917
1,9.884560,56.912140,2017,6.0,12.0,2815,59,2,5690,2935,...,2890,2746,7286,792,376,201,2283,1336,2076,1653
2,8.256020,55.637050,2019,10.0,5.0,2821,26,1,5150,2910,...,2888,2759,8546,1117,372,295,2986,1438,2448,1838
3,-0.402590,43.505630,2018,116.0,14.0,2870,90,3,5315,2992,...,2938,2802,10719,1163,564,197,3216,1948,1948,2926
4,-0.517360,45.806430,2017,84.0,14.0,2858,81,3,5285,2982,...,2927,2792,9353,1095,503,249,3186,1638,1638,2677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80084,10.321640,57.631110,2019,11.0,9.0,2818,24,1,5223,2908,...,2886,2756,7611,960,396,241,2488,1317,2118,1582
80085,10.327990,57.305850,2018,23.0,12.0,2817,30,1,5398,2913,...,2887,2752,8000,919,408,218,2505,1425,2187,1723
80086,15.003900,55.090170,2017,138.0,1.0,2812,19,1,5540,2909,...,2888,2748,6932,781,314,240,2242,1214,2029,1498
80087,17.252948,53.901434,2021,131.0,1.0,2808,66,2,6775,2945,...,2895,2722,7301,816,390,211,2357,1336,2357,1558


In [11]:
_[1]

,speciesId,surveyId
276,5900,729
288,9368,729
287,53,729
286,10255,729
285,10600,729
...,...,...
1483598,3067,3919620
1483597,4077,3919620
1483596,9816,3919620
1483600,540,3919620


In [12]:
## create scipy sparse matrix, fill by row:
from scipy.sparse import csr_matrix

df_species_tmp = _[1]
n_rows = df_species_tmp.surveyId.nunique()
n_cols = df_species_tmp.speciesId.max()

mat = csr_matrix((n_rows, n_cols), dtype=np.int32)
for row, surveyId in tqdm(enumerate(df_species_tmp.surveyId.unique())):
    tmp_df = df_species_tmp.loc[df_species_tmp.surveyId == surveyId]
    cols = tmp_df.speciesId.values - 1
    ## fill mat:
    mat[row, cols] = 1

0it [00:00, ?it/s]/Users/t.vanderplas/anaconda3/envs/geoclef/lib/python3.10/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
8892it [00:05, 1699.85it/s]


In [14]:
## save mat:
from scipy.sparse import save_npz, load_npz
# save_npz(os.path.join(path_dict['data_folder'], 'species_mat_val.npz'), mat)

In [15]:
species_mat_train = load_npz(os.path.join(path_dict['data_folder'], 'species_mat_train.npz'))
species_mat_val = load_npz(os.path.join(path_dict['data_folder'], 'species_mat_val.npz'))

In [16]:
## PCA on sparse matrix:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
pca.fit(species_mat_train)
np.cumsum(pca.explained_variance_ratio_)

TypeError: PCA only support sparse inputs with the "arpack" solver, while "auto" was passed. See TruncatedSVD for a possible alternative.

In [64]:
## PCA analysis:
from sklearn.decomposition import PCA

pca = PCA(n_components=df_train.shape[1] - 5)
pca.fit(df_train.values[:, 5:])
np.cumsum(pca.explained_variance_ratio_)

array([0.88869272, 0.93931589, 0.98060486, 0.99561094, 0.99816131,
       0.99919504, 0.99945668, 0.99964582, 0.99977514, 0.99986259,
       0.99992412, 0.99996675, 0.99999886, 0.99999963, 0.99999991,
       0.99999996, 0.99999999, 1.        , 1.        ])

In [65]:
pca.components_[0, :]

array([ 4.19426015e-05,  2.72773297e-03,  8.60171783e-05,  3.04650027e-02,
        1.28770810e-03, -2.32489870e-03,  3.61260680e-03, -6.01410883e-03,
        5.78857247e-03,  3.93836333e-04, -6.09140311e-04,  8.81682591e-01,
        1.19352046e-01,  3.19003555e-02,  4.78515983e-03,  3.28064146e-01,
        1.22016428e-01,  1.08585970e-01,  2.68364451e-01])

In [66]:
df_train.columns

Index(['lng', 'lat', 'year', 'Elevation', 'LandCover', 'Bio1', 'Bio2', 'Bio3',
       'Bio4', 'Bio5', 'Bio6', 'Bio7', 'Bio8', 'Bio9', 'Bio10', 'Bio11',
       'Bio12', 'Bio13', 'Bio14', 'Bio15', 'Bio16', 'Bio17', 'Bio18', 'Bio19'],
      dtype='object')